## Tenant Volume and EPS By Month
This python notebook leverages the Stellar Cyber API to calculate volume (GB) and EPS (given an event size estimate) for each tenant per month. This notebook can be modified to export to CSV or can be adapted into a script. 

Before using you will have to update the parameters to include the Stellar Cyber URL and API credentials

In [ ]:
import json
import requests
import pandas as pd

In [ ]:
# PARAMETERS
customer_name = '' # does not affect query, just a string written to output
url = '' # Instance url, e.g. customer.stellarcyber.ai
username = '' # API username
api_key = '' # API key
start_date = '2022-02-01T00:00:00.000Z'
end_date = '2023-02-01T00:00:00.000Z'
event_size_bytes = 2000 # Assumption of bytes for every record ingested

In [ ]:
eps_query = {
  "aggs": {
    "month": {
      "date_histogram": {
        "field": "timestamp",
        "calendar_interval": "1M",
        "time_zone": "US/Eastern",
        "min_doc_count": 1
      },
      "aggs": {
        "tenant": {
          "terms": {
            "field": "tenant_name.keyword",
            "order": {
              "avg_daily_throughput": "desc"
            },
            "size": 100
          },
          "aggs": {
            "avg_daily_throughput": {
              "avg": {
                "field": "throughput"
              }
            }
          }
        }
      }
    }
  },
  "size": 0,
  "stored_fields": [
    "*"
  ],
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "match_all": {}
        },
        {
          "range": {
            "timestamp": {
              "gte": start_date,
              "lte": end_date,
              "format": "strict_date_optional_time"
            }
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  }
}

In [ ]:
query = {
    'query': eps_query,
    'index': 'aella-meta*'
}

In [ ]:
headers = {'Accept': 'application/json', 'Content-type': 'application/json'}

In [ ]:
api_url ='https://{}/connect/api/data/{}/_search'.format(url, query['index'])
response = requests.get(api_url, data=json.dumps(query['query']), auth=(username, api_key), headers = headers, verify=False)
response_dict = response.json()

In [ ]:
extracted_list = []
for row in response_dict['aggregations']['month']['buckets']:
      nested_dict_key = None
      nested_dict = {}
      non_nested_dict = {'customer': customer_name}
      for key, val in row.items():
        if type(val) is dict and 'buckets' in val:
          nested_dict_key = key
          nested_dict = val
        else:
          if type(val) is dict and 'value' in val:
            non_nested_dict[key] = val['value']
          else:
            non_nested_dict[key] = val

      # Now go through nested
      if nested_dict_key:
        for nested_row in nested_dict['buckets']:
          extracted_row = non_nested_dict.copy()
          for nested_key, nested_val in nested_row.items():
            if type(nested_val) is dict and 'value' in nested_val:
              extracted_row[nested_key] = nested_val['value']
            else:
              if nested_key == 'key':
                extracted_row[nested_dict_key] = nested_val
              else:
                extracted_row[nested_key] = nested_val
          extracted_list.append(extracted_row)

extracted_pd = pd.DataFrame(extracted_list)

In [ ]:
extracted_pd = extracted_pd.rename(columns={'avg_daily_throughput': 'avg_daily_volume_gb'})
extracted_pd['eps'] = extracted_pd['avg_daily_volume_gb']*1000*1000*1000/event_size_bytes/24/60/60 # Conversion of GB/day to EPS with assumption defined above
extracted_pd['month'] = pd.to_datetime(extracted_pd['key_as_string'], format='%Y-%m-%dT%H:%M:%S.%f%z')
extracted_pd = extracted_pd.drop(['key', 'key_as_string', 'doc_count'], axis=1)

In [ ]:
extracted_pd.tail(10)